# Fine-Tuning InceptionV3 on StateFarm distracted driver dataset

In [14]:
import numpy as np
from keras.layers import *
from keras.models import Model
from keras.optimizers import *
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.utils import np_utils
from keras.initializers import glorot_uniform
import cv2

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Dataset taken from [https://www.kaggle.com/c/state-farm-distracted-driver-detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection)

In [15]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

train_generator = train_datagen.flow_from_directory('train', target_size = (150, 150), batch_size=32, class_mode='categorical')

Found 22424 images belonging to 10 classes.


In [16]:
train_generator.class_indices

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}

In [17]:
base_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

In [18]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

model = Model(base_model.input, x)


In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [21]:
model.fit_generator(train_generator, steps_per_epoch=22424/32, epochs =1, shuffle=True)

Epoch 1/1
701/700 [==============================] - 2040s 3s/step - loss: 1.8074 - acc: 0.3912


In [22]:
for i, layer in enumerate(model.layers):
         print(i, layer.name)

(0, 'input_2')
(1, 'conv2d_95')
(2, 'batch_normalization_95')
(3, 'activation_95')
(4, 'conv2d_96')
(5, 'batch_normalization_96')
(6, 'activation_96')
(7, 'conv2d_97')
(8, 'batch_normalization_97')
(9, 'activation_97')
(10, 'max_pooling2d_5')
(11, 'conv2d_98')
(12, 'batch_normalization_98')
(13, 'activation_98')
(14, 'conv2d_99')
(15, 'batch_normalization_99')
(16, 'activation_99')
(17, 'max_pooling2d_6')
(18, 'conv2d_103')
(19, 'batch_normalization_103')
(20, 'activation_103')
(21, 'conv2d_101')
(22, 'conv2d_104')
(23, 'batch_normalization_101')
(24, 'batch_normalization_104')
(25, 'activation_101')
(26, 'activation_104')
(27, 'average_pooling2d_10')
(28, 'conv2d_100')
(29, 'conv2d_102')
(30, 'conv2d_105')
(31, 'conv2d_106')
(32, 'batch_normalization_100')
(33, 'batch_normalization_102')
(34, 'batch_normalization_105')
(35, 'batch_normalization_106')
(36, 'activation_100')
(37, 'activation_102')
(38, 'activation_105')
(39, 'activation_106')
(40, 'mixed0')
(41, 'conv2d_110')
(42, 'batc

In [23]:
for layer in model.layers[:165]:
        layer.trainable = False
for layer in model.layers[165:]:
        layer.trainable = True

In [24]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [25]:
model.fit_generator(train_generator, steps_per_epoch=22424/32, epochs = 2, shuffle=True)

Epoch 1/2
701/700 [==============================] - 3355s 5s/step - loss: 0.7506 - acc: 0.8332
Epoch 2/2
701/700 [==============================] - 3353s 5s/step - loss: 0.1240 - acc: 0.9649
